# 3.4.0_Dialogue

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import sqlite3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Building Chatbots in Python'
data = make_dir_line(modality, project)

raw = data('raw')

## 3.4.2 Form filling

No he construido la funcion send_message

In [4]:
# # Define the INIT state
# INIT = 0

# # Define the CHOOSE_COFFEE state
# CHOOSE_COFFEE = 1

# # Define the ORDERED state
# ORDERED = 2

# # Define the policy rules
# policy = {
#     (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
#     (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
#     (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
#     (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
# }

# # Create the list of messages
# messages = [
#     "I'd like to become a professional dancer",
#     "well then I'd like to order some coffee",
#     "my favourite animal is a zebra",
#     "kenyan"
# ]

# # Call send_message() for each message
# state = INIT
# for message in messages:    
#     state = send_message(policy, state, message)

Respuesta:

    USER : I'd like to become a professional dancer
    BOT : I'm sorry - I'm not sure how to help you
    USER : well then I'd like to order some coffee
    BOT : ok, Colombian or Kenyan?
    USER : my favourite animal is a zebra
    BOT : I'm sorry - would you like Colombian or Kenyan?
    USER : kenyan
    BOT : perfect, the beans are on their way!

## 3.4.3 Asking contextual questions

In [5]:
# # Define the states
# INIT=0 
# CHOOSE_COFFEE=1
# ORDERED=2

# # Define the policy rules dictionary
# policy_rules = {
#     (INIT, "ask_explanation"): (INIT, "I'm a bot to help you order coffee beans"),
#     (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
#     (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
#     (CHOOSE_COFFEE, "ask_explanation"): (CHOOSE_COFFEE, "We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.")    
# }

# # Define send_messages()
# def send_messages(messages):
#     state = INIT
#     for msg in messages:
#         state = send_message(state, msg)

# # Send the messages
# send_messages([
#     "what can you do for me?",
#     "well then I'd like to order some coffee",
#     "what do you mean by that?",
#     "kenyan"
# ])

Respuesta:

    USER : what can you do for me?
    BOT : I'm a bot to help you order coffee beans
    USER : well then I'd like to order some coffee
    BOT : ok, Colombian or Kenyan?
    USER : what do you mean by that?
    BOT : We have two kinds of coffee beans - the Kenyan ones make a slightly sweeter coffee, and cost $6. The Brazilian beans make a nutty coffee and cost $5.
    USER : kenyan
    BOT : perfect, the beans are on their way!

## 3.4.4 Dealing with rejection

In [6]:
# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " and ".join(filters)
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect(raw / 'hotels.db')
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query, t)
    # Return the results
    return c.fetchall()

In [7]:
# # Define respond()
# def respond(message, params, prev_suggestions, excluded):
#     # Interpret the message
#     parse_data = interpret(message)
#     # Extract the intent
#     intent = parse_data["intent"]["name"]
#     # Extract the entities
#     entities = parse_data["entities"]
#     # Add the suggestion to the excluded list if intent is "deny"
#     if intent == "deny":
#         excluded.extend(prev_suggestions)
#     # Fill the dictionary with entities
#     for ent in entities:
#         params[ent["entity"]] = str(ent["value"])
#     # Find matching hotels
#     results = [
#         r 
#         for r in find_hotels(params, excluded) 
#         if r[0] not in excluded
#     ]
#     # Extract the suggestions
#     names = [r[0] for r in results]
#     n = min(len(results), 3)
#     suggestions = names[:2]
#     return responses[n].format(*names), params, suggestions, excluded

# # Initialize the empty dictionary and lists
# params, suggestions, excluded = {}, [], []

# # Send the messages
# for message in ["I want a mid range hotel", "no that doesn't work for me"]:
#     print("USER: {}".format(message))
#     response, params, suggestions, excluded = respond(message, params, suggestions, excluded)
#     print("BOT: {}".format(response))

Respuestas:

    USER: I want a mid range hotel
    BOT: Hotel for Dogs is one option, but I know others too :)
    USER: no that doesn't work for me
    BOT: Grand Hotel is one option, but I know others too :)

## 3.4.6 Pending actions I

In [8]:
# Define policy()
def policy(intent):
    # Return "do_pending" if the intent is "affirm"
    if intent == "affirm":
        return "do_pending", None
    # Return "Ok" if the intent is "deny"
    if intent == "deny":
        return "Ok", None
    if intent == "order":
        return "Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?", "Alright, I've ordered that for you!"

## 3.4.7 Pending actions II

In [9]:
# # Define send_message()
# def send_message(pending, message):
#     print("USER : {}".format(message))
#     action, pending_action = policy(interpret(message))
#     if action == 'do_pending' and pending is not None:
#         print("BOT : {}".format(pending))
#     else:
#         print("BOT : {}".format(action))
#     return pending_action
    
# # Define send_messages()
# def send_messages(messages):
#     pending = None
#     for msg in messages:
#         pending = send_message(pending, msg)

# # Send the messages
# send_messages([
#     "I'd like to order some coffee",
#     "ok yes please"
# ])

Respuestas:

    USER : I'd like to order some coffee
    BOT : Unfortunately, the Kenyan coffee is currently out of stock, would you like to order the Brazilian beans?
    USER : ok yes please
    BOT : Alright, I've ordered that for you!

## 3.4.8 Pending state transitions

In [10]:
# # Define the states
# INIT=0
# AUTHED=1
# CHOOSE_COFFEE=2
# ORDERED=3

# # Define the policy rules
# policy_rules = {
#     (INIT, "order"): (INIT, "you'll have to log in first, what's your phone number?", AUTHED),
#     (INIT, "number"): (AUTHED, "perfect, welcome back!", None),
#     (AUTHED, "order"): (CHOOSE_COFFEE, "would you like Colombian or Kenyan?", None),    
#     (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!", None)
# }

# # Define send_messages()
# def send_messages(messages):
#     state = INIT
#     pending = None
#     for msg in messages:
#         state, pending = send_message(state, pending, msg)

# # Send the messages
# send_messages([
#     "I'd like to order some coffee",
#     "555-1234",
#     "kenyan"
# ])

Respuestas:

    USER : I'd like to order some coffee
    BOT : you'll have to log in first, what's your phone number?
    USER : 555-1234
    BOT : perfect, welcome back!
    BOT : would you like Colombian or Kenyan?
    USER : kenyan
    BOT : perfect, the beans are on their way!
    BOT : would you like Colombian or Kenyan?

## 3.4.9 Putting it all together I

In [11]:
# # Define chitchat_response()
# def chitchat_response(message):
#     # Call match_rule()
#     response, phrase = match_rule(eliza_rules, message)
#     # Return none if response is "default"
#     if response == "default":
#         return None
#     if '{0}' in response:
#         # Replace the pronouns of phrase
#         phrase = replace_pronouns(phrase)
#         # Calculate the response
#         response = response.format(phrase)
#     return response

## 3.4.10 Putting it all together II

In [12]:
# # Define send_message()
# def send_message(state, pending, message):
#     print("USER : {}".format(message))
#     response = chitchat_response(message)
#     if response is not None:
#         print("BOT : {}".format(response))
#         return state, None
    
#     # Calculate the new_state, response, and pending_state
#     new_state, response, pending_state = policy_rules[(state, interpret(message))]
#     print("BOT : {}".format(response))
#     if pending is not None:
#         new_state, response, pending_state = policy_rules[pending]
#         print("BOT : {}".format(response))        
#     if pending_state is not None:
#         pending = (pending_state, interpret(message))
#     return new_state, pending

# # Define send_messages()
# def send_messages(messages):
#     state = INIT
#     pending = None
#     for msg in messages:
#         state, pending = send_message(state, pending, msg)

# # Send the messages
# send_messages([
#     "I'd like to order some coffee",
#     "555-12345",
#     "do you remember when I ordered 1000 kilos by accident?",
#     "kenyan"
# ])  

Respuestas:

    USER : I'd like to order some coffee
    BOT : you'll have to log in first, what's your phone number?
    USER : 555-12345
    BOT : perfect, welcome back!
    BOT : would you like Colombian or Kenyan?
    USER : do you remember when I ordered 1000 kilos by accident?
    BOT : Yes .. and?
    USER : kenyan
    BOT : perfect, the beans are on their way!

## 3.4.12 Generating text with neural networks

In [13]:
# # Feed the seed text into the neural network
# seed = "i'm gonna punch lenny in the back of the"

# # Iterate over the different temperature values
# for temperature in [0.2, 0.5, 1.0, 1.2]:
#     print("\nGenerating text with riskiness : {}\n".format(temperature))
#     # Call the sample_text function
#     print(sample_text(seed, temperature))

Respuestas:
    
    Generating text with riskiness : 0.2
    
    i'm gonna punch lenny in the back of the been a to the on the man to the mother and the father to simpson the father to with the marge in the for the like the fame to the been to the for my bart the don't was in the like the for the father the father a was the father been a say the been to me the do it and the father been to go. i want to the boy i can the from a man to be the for the been a like the father to make my bart of the father <br>
    
    Generating text with riskiness : 0.5
    
    i'm gonna punch lenny in the back of the kin't she change and i'm all better it and the was the fad a drivera it? what i want to did hey, he would you would in your bus who know is the like and this don't are for your this all for your manset the for it a man is on the see the will they want to know i'm are for one start of that and i got the better this is. it whoce and i don't are on the mater stop in the from a for the be your mileat <br>
    
    Generating text with riskiness : 1.0
    
    i'm gonna punch lenny in the back of the to to macks how screath. firl done we wouldn't wil that kill. of this torshmobote since, i know i ord did, can give crika of sintenn prescoam.whover my me after may? there's right. that up. there's ruining isay.oh.solls.nan'h those off point chuncing car your anal medion.hey, are exallies a off while bea dolk of sure, hello, no in her, we'll rundems... i'm eventy taving me to too the letberngonce
    
    Generating text with riskiness : 1.2
    
    i'm gonna punch


In [14]:
print('Ok_')

Ok_
